In [1]:
from fair_util import BMInterface
from fair_grid import BMGridSearch
from fair_bm import BMType
import pandas as pd

import mkl
mkl.set_num_threads(5)

train_df = pd.read_csv('./data/wids_pre/wids_train.csv')
val_df = pd.read_csv('./data/wids_pre/wids_val.csv')
test_df = pd.read_csv('./data/wids_pre/wids_test.csv')

# Defining the name of the label column
label_name = 'DiagPeriodL90D'
# Defininf the name of the privileged group
sensitive_attribute = ['patient_age']

bmI = BMInterface(train_df, val_df, test_df, label_name, sensitive_attribute)

privileged_group = [{'patient_age': 1}]
unprivileged_group = [{'patient_age': 0}]

C:\Users\enemy\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\enemy\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_disc

In [2]:
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC  
from sklearn.linear_model import LogisticRegression
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier

mlp= MLPClassifier(
    max_iter=500,
    hidden_layer_sizes = (200,200,200,200),
    activation = 'logistic',
    solver = 'adam',
    alpha = 0.01,
    learning_rate = 'adaptive'
)

# Create a new instance of the classifier
lr = LogisticRegression(solver='liblinear')

xgbr =  XGBClassifier(
    max_depth=8,
    learning_rate=0.01,
    gamma = 0.25,
    n_estimators = 500,
    subsample = 0.8,
    colsample_bytree = 0.3)

tabnet = TabNetClassifier()

cat = CatBoostClassifier(eval_metric='Accuracy',
                         depth =  4,
                         learning_rate = 0.01,
                         iterations = 10)


model_list = [tabnet, xgbr, cat, mlp, lr]

C:\Users\enemy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
tabnet.__str__()#.startswith('XGBClassifier')

In [5]:
from itertools import product
from more_itertools import powerset

pre_bm = [BMType.preReweighing,BMType.preDisparate]
pos_bm = [BMType.posCalibrated, BMType.posEqqOds, BMType.posROC]

bm_list = [[pre, pos] for pre, pos in product(pre_bm, pos_bm)]
bm_list.extend([p] for p in pre_bm); bm_list.extend([p] for p in pos_bm)
bm_list

[[<BMType.preReweighing: 1>, <BMType.posCalibrated: 6>],
 [<BMType.preReweighing: 1>, <BMType.posEqqOds: 7>],
 [<BMType.preReweighing: 1>, <BMType.posROC: 8>],
 [<BMType.preDisparate: 2>, <BMType.posCalibrated: 6>],
 [<BMType.preDisparate: 2>, <BMType.posEqqOds: 7>],
 [<BMType.preDisparate: 2>, <BMType.posROC: 8>],
 [<BMType.preReweighing: 1>],
 [<BMType.preDisparate: 2>],
 [<BMType.posCalibrated: 6>],
 [<BMType.posEqqOds: 7>],
 [<BMType.posROC: 8>]]

In [7]:
import copy

bm_list_in = copy.deepcopy(bm_list)
[l.append(BMType.inAdversarial) for l in bm_list_in]
bm_list_in.extend(bm_list)
bm_list_in.append([BMType.inAdversarial])
bm_list_in

[[<BMType.preReweighing: 1>,
  <BMType.posCalibrated: 6>,
  <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.posEqqOds: 7>, <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.posROC: 8>, <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>,
  <BMType.posCalibrated: 6>,
  <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>, <BMType.posEqqOds: 7>, <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>, <BMType.posROC: 8>, <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.inAdversarial: 4>],
 [<BMType.preDisparate: 2>, <BMType.inAdversarial: 4>],
 [<BMType.posCalibrated: 6>, <BMType.inAdversarial: 4>],
 [<BMType.posEqqOds: 7>, <BMType.inAdversarial: 4>],
 [<BMType.posROC: 8>, <BMType.inAdversarial: 4>],
 [<BMType.preReweighing: 1>, <BMType.posCalibrated: 6>],
 [<BMType.preReweighing: 1>, <BMType.posEqqOds: 7>],
 [<BMType.preReweighing: 1>, <BMType.posROC: 8>],
 [<BMType.preDisparate: 2>, <BMType.posCalibrated: 6>],
 [<BMType.preDisparate: 

In [ ]:
for model in model_list:
    bmG = BMGridSearch(bmI, model=model, bm_list=bm_list, privileged_group=privileged_group, unprivileged_group=unprivileged_group)
    bmG.run_single_sensitive()

epoch 0  | loss: 0.71386 | val_0_auc: 0.53637 |  0:00:02s
epoch 1  | loss: 0.65379 | val_0_auc: 0.60158 |  0:00:03s
epoch 2  | loss: 0.62191 | val_0_auc: 0.68005 |  0:00:05s
epoch 3  | loss: 0.57401 | val_0_auc: 0.74398 |  0:00:07s
epoch 4  | loss: 0.53738 | val_0_auc: 0.76264 |  0:00:10s
epoch 5  | loss: 0.51833 | val_0_auc: 0.76603 |  0:00:12s
epoch 6  | loss: 0.50645 | val_0_auc: 0.77464 |  0:00:16s
epoch 7  | loss: 0.50022 | val_0_auc: 0.77466 |  0:00:19s
epoch 8  | loss: 0.49713 | val_0_auc: 0.78409 |  0:00:21s
epoch 9  | loss: 0.48971 | val_0_auc: 0.78003 |  0:00:23s
epoch 10 | loss: 0.49002 | val_0_auc: 0.78226 |  0:00:24s
epoch 11 | loss: 0.48315 | val_0_auc: 0.78805 |  0:00:26s
epoch 12 | loss: 0.48263 | val_0_auc: 0.78561 |  0:00:27s
epoch 13 | loss: 0.48427 | val_0_auc: 0.78555 |  0:00:29s
epoch 14 | loss: 0.48122 | val_0_auc: 0.78462 |  0:00:30s
epoch 15 | loss: 0.48054 | val_0_auc: 0.78781 |  0:00:31s
epoch 16 | loss: 0.48542 | val_0_auc: 0.78992 |  0:00:33s
epoch 17 | los

C:\Users\enemy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.71386 | val_0_auc: 0.53637 |  0:00:02s
epoch 1  | loss: 0.65379 | val_0_auc: 0.60158 |  0:00:05s
epoch 2  | loss: 0.62191 | val_0_auc: 0.68005 |  0:00:07s
epoch 3  | loss: 0.57401 | val_0_auc: 0.74398 |  0:00:09s
epoch 4  | loss: 0.53738 | val_0_auc: 0.76264 |  0:00:12s
epoch 5  | loss: 0.51833 | val_0_auc: 0.76603 |  0:00:14s
epoch 6  | loss: 0.50645 | val_0_auc: 0.77464 |  0:00:16s
epoch 7  | loss: 0.50022 | val_0_auc: 0.77466 |  0:00:19s
epoch 8  | loss: 0.49713 | val_0_auc: 0.78409 |  0:00:21s
epoch 9  | loss: 0.48971 | val_0_auc: 0.78003 |  0:00:24s
epoch 10 | loss: 0.49002 | val_0_auc: 0.78226 |  0:00:26s
epoch 11 | loss: 0.48315 | val_0_auc: 0.78805 |  0:00:29s
epoch 12 | loss: 0.48263 | val_0_auc: 0.78561 |  0:00:32s
epoch 13 | loss: 0.48427 | val_0_auc: 0.78555 |  0:00:34s
epoch 14 | loss: 0.48122 | val_0_auc: 0.78462 |  0:00:37s
epoch 15 | loss: 0.48054 | val_0_auc: 0.78781 |  0:00:39s
epoch 16 | loss: 0.48542 | val_0_auc: 0.78992 |  0:00:42s
epoch 17 | los

C:\Users\enemy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.71386 | val_0_auc: 0.53637 |  0:00:02s
epoch 1  | loss: 0.65379 | val_0_auc: 0.60158 |  0:00:04s
epoch 2  | loss: 0.62191 | val_0_auc: 0.68005 |  0:00:06s
epoch 3  | loss: 0.57401 | val_0_auc: 0.74398 |  0:00:09s
epoch 4  | loss: 0.53738 | val_0_auc: 0.76264 |  0:00:11s
epoch 5  | loss: 0.51833 | val_0_auc: 0.76603 |  0:00:14s
epoch 6  | loss: 0.50645 | val_0_auc: 0.77464 |  0:00:16s
epoch 7  | loss: 0.50022 | val_0_auc: 0.77466 |  0:00:19s
epoch 8  | loss: 0.49713 | val_0_auc: 0.78409 |  0:00:22s
epoch 9  | loss: 0.48971 | val_0_auc: 0.78003 |  0:00:25s
epoch 10 | loss: 0.49002 | val_0_auc: 0.78226 |  0:00:27s
epoch 11 | loss: 0.48315 | val_0_auc: 0.78805 |  0:00:30s
epoch 12 | loss: 0.48263 | val_0_auc: 0.78561 |  0:00:33s
epoch 13 | loss: 0.48427 | val_0_auc: 0.78555 |  0:00:36s
epoch 14 | loss: 0.48122 | val_0_auc: 0.78462 |  0:00:40s
epoch 15 | loss: 0.48054 | val_0_auc: 0.78781 |  0:00:45s
epoch 16 | loss: 0.48542 | val_0_auc: 0.78992 |  0:00:49s
epoch 17 | los

...

epoch 0  | loss: 0.71711 | val_0_auc: 0.50743 |  0:00:02s
epoch 1  | loss: 0.65092 | val_0_auc: 0.61697 |  0:00:05s
epoch 2  | loss: 0.60893 | val_0_auc: 0.68062 |  0:00:07s
epoch 3  | loss: 0.56115 | val_0_auc: 0.73838 |  0:00:09s
epoch 4  | loss: 0.53806 | val_0_auc: 0.76851 |  0:00:12s
epoch 5  | loss: 0.53458 | val_0_auc: 0.7784  |  0:00:14s
epoch 6  | loss: 0.52729 | val_0_auc: 0.7829  |  0:00:16s
epoch 7  | loss: 0.51568 | val_0_auc: 0.77515 |  0:00:18s
epoch 8  | loss: 0.51145 | val_0_auc: 0.77915 |  0:00:20s
epoch 9  | loss: 0.50622 | val_0_auc: 0.77856 |  0:00:22s
epoch 10 | loss: 0.50618 | val_0_auc: 0.77602 |  0:00:24s
epoch 11 | loss: 0.49876 | val_0_auc: 0.77481 |  0:00:27s
epoch 12 | loss: 0.49856 | val_0_auc: 0.77522 |  0:00:29s
epoch 13 | loss: 0.49655 | val_0_auc: 0.77942 |  0:00:31s
epoch 14 | loss: 0.49517 | val_0_auc: 0.77635 |  0:00:33s
epoch 15 | loss: 0.49098 | val_0_auc: 0.77809 |  0:00:36s
epoch 16 | loss: 0.49038 | val_0_auc: 0.77401 |  0:00:38s

Early stoppin

C:\Users\enemy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.71711 | val_0_auc: 0.50743 |  0:00:02s
epoch 1  | loss: 0.65092 | val_0_auc: 0.61697 |  0:00:06s
epoch 2  | loss: 0.60893 | val_0_auc: 0.68062 |  0:00:08s
epoch 3  | loss: 0.56115 | val_0_auc: 0.73838 |  0:00:12s
epoch 4  | loss: 0.53806 | val_0_auc: 0.76851 |  0:00:14s
epoch 5  | loss: 0.53458 | val_0_auc: 0.7784  |  0:00:16s
epoch 6  | loss: 0.52729 | val_0_auc: 0.7829  |  0:00:18s
epoch 7  | loss: 0.51568 | val_0_auc: 0.77515 |  0:00:20s
epoch 8  | loss: 0.51145 | val_0_auc: 0.77915 |  0:00:23s
epoch 9  | loss: 0.50622 | val_0_auc: 0.77856 |  0:00:25s
epoch 10 | loss: 0.50618 | val_0_auc: 0.77602 |  0:00:27s
epoch 11 | loss: 0.49876 | val_0_auc: 0.77481 |  0:00:30s
epoch 12 | loss: 0.49856 | val_0_auc: 0.77522 |  0:00:33s
epoch 13 | loss: 0.49655 | val_0_auc: 0.77942 |  0:00:36s
epoch 14 | loss: 0.49517 | val_0_auc: 0.77635 |  0:00:38s
epoch 15 | loss: 0.49098 | val_0_auc: 0.77809 |  0:00:40s
epoch 16 | loss: 0.49038 | val_0_auc: 0.77401 |  0:00:43s

Early stoppin

C:\Users\enemy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\enemy\anaconda3\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


epoch 0  | loss: 0.71386 | val_0_auc: 0.53637 |  0:00:01s
epoch 1  | loss: 0.65379 | val_0_auc: 0.60158 |  0:00:03s
epoch 2  | loss: 0.62191 | val_0_auc: 0.68005 |  0:00:05s
epoch 3  | loss: 0.57401 | val_0_auc: 0.74398 |  0:00:08s
epoch 4  | loss: 0.53738 | val_0_auc: 0.76264 |  0:00:11s
epoch 5  | loss: 0.51833 | val_0_auc: 0.76603 |  0:00:15s
epoch 6  | loss: 0.50645 | val_0_auc: 0.77464 |  0:00:18s
epoch 7  | loss: 0.50022 | val_0_auc: 0.77466 |  0:00:20s
epoch 8  | loss: 0.49713 | val_0_auc: 0.78409 |  0:00:22s
epoch 9  | loss: 0.48971 | val_0_auc: 0.78003 |  0:00:24s
epoch 10 | loss: 0.49002 | val_0_auc: 0.78226 |  0:00:26s
epoch 11 | loss: 0.48315 | val_0_auc: 0.78805 |  0:00:29s
epoch 12 | loss: 0.48263 | val_0_auc: 0.78561 |  0:00:31s
epoch 13 | loss: 0.48427 | val_0_auc: 0.78555 |  0:00:33s
epoch 14 | loss: 0.48122 | val_0_auc: 0.78462 |  0:00:35s
epoch 15 | loss: 0.48054 | val_0_auc: 0.78781 |  0:00:37s
epoch 16 | loss: 0.48542 | val_0_auc: 0.78992 |  0:00:39s
epoch 17 | los

In [13]:
bmG = BMGridSearch(bmI, model=None, bm_list=[[BMType.inAdversarial]], privileged_group=privileged_group, unprivileged_group=unprivileged_group)
bmG.run_single_sensitive()

epoch 0; iter: 0; batch classifier loss: 0.745372
epoch 1; iter: 0; batch classifier loss: 0.612154
epoch 2; iter: 0; batch classifier loss: 0.635546
epoch 3; iter: 0; batch classifier loss: 0.565819
epoch 4; iter: 0; batch classifier loss: 0.600401
epoch 5; iter: 0; batch classifier loss: 0.577236
epoch 6; iter: 0; batch classifier loss: 0.567886
epoch 7; iter: 0; batch classifier loss: 0.506691
epoch 8; iter: 0; batch classifier loss: 0.465262
epoch 9; iter: 0; batch classifier loss: 0.546089
epoch 10; iter: 0; batch classifier loss: 0.468253
epoch 11; iter: 0; batch classifier loss: 0.522293
epoch 12; iter: 0; batch classifier loss: 0.659548
epoch 13; iter: 0; batch classifier loss: 0.436789
epoch 14; iter: 0; batch classifier loss: 0.508616
epoch 15; iter: 0; batch classifier loss: 0.490434
epoch 16; iter: 0; batch classifier loss: 0.357344
epoch 17; iter: 0; batch classifier loss: 0.367088
epoch 18; iter: 0; batch classifier loss: 0.472829
epoch 19; iter: 0; batch classifier loss:

In [12]:
for bm in bm_list:
    bmG = BMGridSearch(bmI, model=None, bm_list=bm_list_in, privileged_group=privileged_group, unprivileged_group=unprivileged_group)
    bmG.run_single_sensitive()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

epoch 0; iter: 0; batch classifier loss: 0.652434
epoch 1; iter: 0; batch classifier loss: 0.502002
epoch 2; iter: 0; batch classifier loss: 0.561557
epoch 3; iter: 0; batch classifier loss: 0.491509
epoch 4; iter: 0; batch classifier loss: 0.528523
epoch 5; iter: 0; batch classifier loss: 0.446660
epoch 6; iter: 0; batch classifier loss: 0.542133
epoch 7; iter: 0; batch classifier loss: 0.464945
epoch 8; iter: 0; batch classifier loss: 0.520321
epoch 9; iter: 0; batch classifier loss: 0.387188
epoch 10; iter: 0; batch classifier loss: 0.440515
epoch 11; iter: 0; batch classifier loss: 0.409837
epoch 12; iter: 0; batch classifier loss: 0.497330
epoch 13; iter: 0; batch classifier loss: 0.350966
epoch 14; iter: 0; batch classifier loss: 0.427014
epoch 15; iter: 0; batch classifier loss: 0.433332
epoch 16; iter: 0; batch classifier loss: 0.588257
epoch 17; iter: 0; batch c

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


epoch 0; iter: 0; batch classifier loss: 0.761945; batch adversarial loss: 0.836422
epoch 1; iter: 0; batch classifier loss: 0.464835; batch adversarial loss: 0.818520
epoch 2; iter: 0; batch classifier loss: 0.520592; batch adversarial loss: 0.649080
epoch 3; iter: 0; batch classifier loss: 0.591772; batch adversarial loss: 0.515216
epoch 4; iter: 0; batch classifier loss: 0.608665; batch adversarial loss: 0.399783
epoch 5; iter: 0; batch classifier loss: 0.641985; batch adversarial loss: 0.412909
epoch 6; iter: 0; batch classifier loss: 0.900737; batch adversarial loss: 0.372083
epoch 7; iter: 0; batch classifier loss: 1.058722; batch adversarial loss: 0.343683
epoch 8; iter: 0; batch classifier loss: 0.965169; batch adversarial loss: 0.325842
epoch 9; iter: 0; batch classifier loss: 1.129415; batch adversarial loss: 0.328122
epoch 10; iter: 0; batch classifier loss: 0.794934; batch adversarial loss: 0.253463
epoch 11; iter: 0; batch classifier loss: 1.043097; batch adversarial loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


epoch 0; iter: 0; batch classifier loss: 0.715142; batch adversarial loss: 0.502135
epoch 1; iter: 0; batch classifier loss: 1.209981; batch adversarial loss: 0.645674
epoch 2; iter: 0; batch classifier loss: 1.181699; batch adversarial loss: 0.597316
epoch 3; iter: 0; batch classifier loss: 1.429844; batch adversarial loss: 0.507139
epoch 4; iter: 0; batch classifier loss: 1.437580; batch adversarial loss: 0.450759
epoch 5; iter: 0; batch classifier loss: 1.606672; batch adversarial loss: 0.402746
epoch 6; iter: 0; batch classifier loss: 1.985302; batch adversarial loss: 0.421620
epoch 7; iter: 0; batch classifier loss: 2.455184; batch adversarial loss: 0.337903
epoch 8; iter: 0; batch classifier loss: 2.143671; batch adversarial loss: 0.347339
epoch 9; iter: 0; batch classifier loss: 2.316683; batch adversarial loss: 0.277795
epoch 10; iter: 0; batch classifier loss: 1.819189; batch adversarial loss: 0.374337
epoch 11; iter: 0; batch classifier loss: 2.550350; batch adversarial loss:

...

epoch 0; iter: 0; batch classifier loss: 0.833803; batch adversarial loss: 0.882705
epoch 1; iter: 0; batch classifier loss: 0.509233; batch adversarial loss: 0.901781
epoch 2; iter: 0; batch classifier loss: 0.451895; batch adversarial loss: 0.725384
epoch 3; iter: 0; batch classifier loss: 0.630635; batch adversarial loss: 0.572326
epoch 4; iter: 0; batch classifier loss: 0.639754; batch adversarial loss: 0.463087
epoch 5; iter: 0; batch classifier loss: 0.664611; batch adversarial loss: 0.386762
epoch 6; iter: 0; batch classifier loss: 0.587131; batch adversarial loss: 0.355559
epoch 7; iter: 0; batch classifier loss: 0.861760; batch adversarial loss: 0.337501
epoch 8; iter: 0; batch classifier loss: 0.888261; batch adversarial loss: 0.356988
epoch 9; iter: 0; batch classifier loss: 0.882123; batch adversarial loss: 0.297259
epoch 10; iter: 0; batch classifier loss: 0.993273; batch adversarial loss: 0.264308
epoch 11; iter: 0; batch classifier loss: 0.872625; batch adversarial loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


epoch 0; iter: 0; batch classifier loss: 0.705612; batch adversarial loss: 0.682523
epoch 1; iter: 0; batch classifier loss: 1.007221; batch adversarial loss: 0.618191
epoch 2; iter: 0; batch classifier loss: 1.228735; batch adversarial loss: 0.557998
epoch 3; iter: 0; batch classifier loss: 1.326683; batch adversarial loss: 0.506600
epoch 4; iter: 0; batch classifier loss: 1.619249; batch adversarial loss: 0.472921
epoch 5; iter: 0; batch classifier loss: 1.879376; batch adversarial loss: 0.417373
epoch 6; iter: 0; batch classifier loss: 2.030931; batch adversarial loss: 0.397558
epoch 7; iter: 0; batch classifier loss: 2.277358; batch adversarial loss: 0.366408
epoch 8; iter: 0; batch classifier loss: 2.265549; batch adversarial loss: 0.316494
epoch 9; iter: 0; batch classifier loss: 2.261204; batch adversarial loss: 0.290558
epoch 10; iter: 0; batch classifier loss: 1.897865; batch adversarial loss: 0.340880
epoch 11; iter: 0; batch classifier loss: 2.215672; batch adversarial loss:

C:\Users\enemy\anaconda3\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


MemoryError: bad allocation